# AgentCore Data Science with Neo4j MCP

This notebook demonstrates how to build a **Native Python MCP Server** for data science workflows, test it locally, and deploy it to **AWS AgentCore**.

We will use the **Neo4j Demo Database** (ReadOnly) containing company and news data.

## Prerequisites

Ensure you have an AWS account with access to Amazon Bedrock AgentCore if you plan to deploy.


In [ ]:
!pip install mcp neo4j boto3

## 1. Environment Setup

Connect to the public Neo4j Demo Database.

In [ ]:
import os

# Public Demo Database Credentials
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com:7687"
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"
os.environ["NEO4J_DATABASE"] = "companies"

print("Environment variables set.")

## 2. Define MCP Server

We use `FastMCP` to define tools that querying the graph.

In [ ]:
from mcp.server.fastmcp import FastMCP
from neo4j import GraphDatabase

# Initialize FastMCP
mcp = FastMCP("Company Research Agent")

@mcp.tool()
def query_company(name: str) -> str:
    """Get company details, industries, and recent news from the knowledge graph."""
    
    query = """
    MATCH (c:Organization {name: $name})
    OPTIONAL MATCH (c)-[:IN_INDUSTRY]->(i:Industry)
    OPTIONAL MATCH (c)<-[:MENTIONS]-(a:Article)
    RETURN c.name as name, 
           collect(distinct i.name) as industries,
           collect(distinct a.title)[..5] as recent_news
    """
    
    uri = os.environ["NEO4J_URI"]
    auth = (os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"])
    
    with GraphDatabase.driver(uri, auth=auth) as driver:
        result = driver.execute_query(
            query, 
            name=name,
            database_=os.environ["NEO4J_DATABASE"]
        )
        records = result.records
        
        if not records:
            return f"No data found for company: {name}"
            
        return str(records[0].data())

print("MCP Server defined.")

## 3. Local Simulation

We can test the tool locally without deploying to AWS yet.

In [ ]:
# Simulate a tool call
result = query_company(name="Google")
print(result)

## 4. Deploy to AWS AgentCore

To deploy this as a Native Agent, you would typically save the python code to a file (e.g., `server.py`) and use the `agentcore` CLI.

```bash
# 1. Save code to server.py
# 2. Deploy
agentcore configure -e server.py --protocol MCP
```

### Invoking the Agent remotely

Once deployed, you can invoke it using `boto3`.

In [ ]:
import boto3
import json

# Initialize Bedrock Agent Runtime Client
# NOTE: This requires valid AWS credentials and a deployed agent
try:
    client = boto3.client('bedrock-agent-runtime')
    
    # Example invocation (Replace parameters with actual values)
    # response = client.invoke_agent(
    #     agentId='YOUR_AGENT_ID',
    #     agentAliasId='YOUR_ALIAS_ID',
    #     sessionId='test-session-1',
    #     inputText='Tell me about Google'
    # )
    
    # print(response['completion'])
    print("Boto3 client initialized. Uncomment code above to invoke actual agent.")
    
except Exception as e:
    print(f"Could not initialize boto3 client (expected if no creds): {e}")